------------------

In [ ]:
!pip install langchain langchain-community jq langchain-huggingface faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import JSONLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import gc
import os
import time
import psutil
import pickle
import torch
import json

In [6]:
# Define file paths
json_file = "/content/drive/MyDrive/cleaned_wiki_articles.json"
output_pickle = "/content/drive/MyDrive/split_documents.pkl"

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# Load JSON data
with open(json_file, 'r') as f:
    data = json.load(f)

# Initialize storage for split documents
split_documents = []

# Process topics and chunk the documents
print("Processing and chunking documents...")
for topic, articles in tqdm(data.items(), desc="Processing topics"):
    for article in tqdm(articles, desc=f"Processing topic: {topic}"):
        # Extract relevant fields
        title = article.get("title", "Unknown Title")
        url = article.get("url", "No URL")
        summary = article.get("summary", "")
        metadata = {
            "topic": topic,
            "title": title,
            "url": url,
        }

        # Chunk the text
        chunks = text_splitter.split_text(summary)
        split_documents.extend([
            Document(
                page_content=chunk,
                metadata=metadata
            ) for chunk in chunks
        ])

    # Free memory after each topic
    gc.collect()

# Save split documents to a pickle file
with open(output_pickle, 'wb') as f:
    pickle.dump(split_documents, f)

print(f"Total chunks created: {len(split_documents)}")
print(f"Chunked documents saved to: {output_pickle}")

Processing and chunking documents...


Processing topics: 100%|██████████| 10/10 [02:43<00:00, 16.32s/it]


Total chunks created: 3437657
Chunked documents saved to: /content/drive/MyDrive/split_documents.pkl


In [8]:
# Function to monitor memory usage
def monitor_memory():
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / 1024 ** 2  # Convert to MB
    print(f"Memory usage: {mem:.2f} MB")

# Load saved chunks
with open('/content/drive/MyDrive/split_documents.pkl', 'rb') as f:
    split_documents = pickle.load(f)

print(f"Loaded {len(split_documents)} document chunks for embedding.")

# Initialize HuggingFace embeddings with GPU support (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2",
    model_kwargs={"device": device}  # Specify the device
)

# Process embeddings in batches with progress tracking
batch_size = 50000  # Adjust as needed
print("Creating embeddings...")
start_time = time.time()

faiss_index = None
for batch_num, start_idx in enumerate(tqdm(range(0, len(split_documents), batch_size), desc="Processing embeddings")):
    end_idx = min(start_idx + batch_size, len(split_documents))
    batch_docs = split_documents[start_idx:end_idx]

    # Extract text and metadata
    batch_texts = [doc.page_content for doc in batch_docs]
    batch_metadatas = [doc.metadata for doc in batch_docs]  # Includes topic, title, and URL

    # If it's the first batch, initialize FAISS
    if faiss_index is None:
        faiss_index = FAISS.from_texts(
            texts=batch_texts,
            embedding=embedding_model,  # Pass the full embedding model here
            metadatas=batch_metadatas,  # Pass metadata
        )
    else:
        # Add embeddings and metadata incrementally
        faiss_index.add_texts(batch_texts, batch_metadatas)

    # Print progress and elapsed time
    elapsed_time = time.time() - start_time
    print(f"Batch {batch_num + 1} processed in {elapsed_time:.2f} seconds.")

    # Free memory after each batch
    del batch_docs, batch_texts, batch_metadatas
    gc.collect()
    monitor_memory()

# Save FAISS index
faiss_index.save_local("/content/drive/MyDrive/faiss_index")


end_time = time.time()
print(f"Embeddings created and saved to FAISS index in {end_time - start_time:.2f} seconds.")

Loaded 3437657 document chunks for embedding.
Using device: cuda


<ipython-input-8-22593b4b9b97>:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

Processing embeddings:  99%|█████████▊| 68/69 [22:44<00:21, 21.72s/it]

Batch 68 processed in 1358.53 seconds.
Memory usage: 14579.33 MB


Processing embeddings: 100%|██████████| 69/69 [23:02<00:00, 20.04s/it]

Batch 69 processed in 1376.32 seconds.
Memory usage: 14635.23 MB


Embeddings created and saved to FAISS index in 1420.77 seconds.


In [9]:
# Load FAISS index from disk
db_folder = "/content/drive/MyDrive/faiss_index"
vectorstore = FAISS.load_local(db_folder, embedding_model, allow_dangerous_deserialization=True)
print("Loaded FAISS index from disk.")

# User query


Loaded FAISS index from disk.


In [12]:
# User query
user_query = input("Enter your question: ")

# Retrieve the most relevant context using Maximum Marginal Relevance
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3, "fetch_k": 4, "lambda_mult": 0.4})

start_time_query = time.time()
results = retriever.get_relevant_documents(user_query)
end_time_query = time.time()

print(f"\nQuery processed in {end_time_query - start_time_query:.4f} seconds.")

# Extract and display context with metadata
if results:
    print("\nRetrieved Results:")
    for idx, result in enumerate(results, start=1):
        print(f"\nResult {idx}:")
        print(f"Context: {result.page_content}")
        print(f"Title: {result.metadata.get('title', 'Unknown Title')}")
        print(f"URL: {result.metadata.get('url', 'No URL')}")
        print(f"Topic: {result.metadata.get('topic', 'Unknown Topic')}")
else:
    print("\nNo relevant context found.")

# Free memory after query processing
del results, retriever
gc.collect()

Enter your question: Yellow fever

Query processed in 0.4555 seconds.

Retrieved Results:

Result 1:
Context: yellow fever is a viral disease of typically short duration in most cases symptoms include fever chills loss of appetite nausea muscle pains particularly in the back and headaches symptoms typically
Title: Yellow fever
URL: https://en.wikipedia.org/wiki/Yellow_fever
Topic: Health

Result 2:
Context: yellow fever is common in tropical and subtropical areas of south america and africa worldwide about 600 million people live in endemic areas the who estimates 200 000 cases of yellow fever worldwide
Title: Yellow fever
URL: https://en.wikipedia.org/wiki/Yellow_fever
Topic: Health

Result 3:
Context: to the yellow fever vaccine is known as yellow fever vaccine associated acute neurotropic disease yel and the canadian medical association published a 2001 cmaj article entitled yellow fever
Title: Yellow fever vaccine
URL: https://en.wikipedia.org/wiki/Yellow_fever_vaccine
Topic: Healt

0